# A játék leírása

A 2023/24-es MatProgCsom tárgy keretein belül egy logikai rejtvény megoldási folyamatát implementáltuk.
A rejtvény szabályai a következők:

*Töltse ki az ábrát "pókokkal" az alábbi szabályok alapján:*

+ *Az ábrában minden pókhálóhoz igyekszik egy pók, ami azt jelenti, hogy a vízszintesen vagy függőlegesen közvetlenül szomszédos mezőben található.*

+ *Pókot tartalmazó mezők nem érintkezhetnek egymással, még átlósan sem.*

+ *Az ábra szélén látható számok azt jelzik, hogy az adott sorban ill. oszlopban hány pók van.*

Például alább látható egy feladvány, illetve ennek helyes kitöltése:

<table><tr>
<td> <img src="example_task.jpg" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="example_sol.jpg" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

# A program

A programmal alapvetően azt igyekeztünk szimulálni, ahogyan egy emberi játékos megoldaná a feladványt: először megpróbáljuk "kilogikázni" a következő lépést, például különböző mintázatokat keresve, ha pedig ezekkel már nem tudunk továbblépni, akkor backtrackingre támaszkodva a DFS algoritmussal keressük a megoldást.

A táblát egy $N\times N$-es NumPy array-jel reprezentáljuk, valamint eltároljuk mellé azt a két vektort, amelyek azt adják meg, hogy az egyes sorokba, illetve oszlopokba hány pókot kell még beírni. A táblázatban az alábbi jelöléseket használjuk:
+ 0: még üres mező
+ 1: kielégítetlen pókháló
+ 2: kielégített pókháló
+ 3: kielégítetlen pók
+ 4: kielégített pók
+ 9: olyan mező, ahova nem kerülhet pók

A terminológia szorul némi magyarázatra:

A játék szabályai előírják, hogy a helyes kitöltésben minden pókhálóhoz kell tartoznia póknak. Ha egy pókhálóra már egyértelmű, hogy melyik vele oldalszomszédos pók tartozik hozzá, akkor őt és pókját *kielégítettnek* nevezzük, addig pedig *kielégítetlennek*.

In [1]:
import numpy as np
import tabulate # a tábla szép megjelenítéséhez

## Segédfüggvények

In [2]:
# A DFS során szükségünk lesz arra, hogy a tábla egy adott állását kompakt módon,
# stringként tudjuk tárolni. Az alábbi függvények az e két reprezentáció közötti átjárást biztosítják.
def table_to_string(t, r, c):
    return ''.join(np.array(list(map(str, np.append(np.append(t, r), c)))))

def string_to_table(s):
    arr = np.array(np.array(list(map(int, [*s])))).reshape(N+2,N)
    return [arr[:N], arr[N], arr[N+1]]

In [3]:
# Visszaadja az egy mezővel oldalszomszédos mezőket {pozíció: érték} párok dictjeként.
def get_4_neighbours(t, i, j):
    if i == 0:
        if j == 0:
            return {(i+1, j): t[i+1, j], (i, j+1): t[i, j+1]}
        if j == N-1:
            return {(i+1, j): t[i+1, j], (i, j-1): t[i, j-1]}
        return {(i+1, j): t[i+1, j], (i, j-1): t[i, j-1], (i, j+1): t[i, j+1]}
    if i == N-1:
        if j == 0:
            return {(i-1, j): t[i-1, j], (i, j+1): t[i, j+1]}
        if j == N-1:
            return {(i-1, j): t[i-1, j], (i, j-1): t[i, j-1]}
        return {(i-1, j): t[i-1, j], (i, j-1): t[i, j-1], (i, j+1): t[i, j+1]}
    if j == 0:
        return {(i-1, j): t[i-1, j], (i+1, j): t[i+1, j], (i, j+1): t[i, j+1]}
    if j == N-1:
        return {(i-1, j): t[i-1, j], (i+1, j): t[i+1, j], (i, j-1): t[i, j-1]}
    return {(i-1, j): t[i-1, j], (i+1, j): t[i+1, j], (i, j-1): t[i, j-1], (i, j+1): t[i, j+1]}

In [4]:
# Visszaadja az egy mezővel (akár átlósan) szomszédos mezőket {pozíció: érték} párok dictjeként.
def get_8_neighbours(t, i, j):
    if i == 0:
        if j == 0:
            return {(i+1, j): t[i+1, j], (i, j+1): t[i, j+1], (i+1, j+1): t[i+1, j+1]}
        if j == N-1:
            return {(i+1, j): t[i+1, j], (i, j-1): t[i, j-1],  (i+1, j-1): t[i+1, j-1]}
        return {(i+1, j): t[i+1, j], (i, j-1): t[i, j-1], (i, j+1): t[i, j+1], (i+1, j-1): t[i+1, j-1], (i+1, j+1): t[i+1, j+1]}
    if i == N-1:
        if j == 0:
            return {(i-1, j): t[i-1, j], (i, j+1): t[i, j+1], (i-1, j+1): t[i-1, j+1]}
        if j == N-1:
            return {(i-1, j): t[i-1, j], (i, j-1): t[i, j-1], (i-1, j-1): t[i-1, j-1]}
        return {(i-1, j): t[i-1, j], (i, j-1): t[i, j-1], (i, j+1): t[i, j+1], (i-1, j-1): t[i-1, j-1], (i-1, j+1): t[i-1, j+1]}
    if j == 0:
        return {(i-1, j): t[i-1, j], (i+1, j): t[i+1, j], (i, j+1): t[i, j+1], (i-1, j+1): t[i-1, j+1], (i+1, j+1): t[i+1, j+1]}
    if j == N-1:
        return {(i-1, j): t[i-1, j], (i+1, j): t[i+1, j], (i, j-1): t[i, j-1], (i-1, j-1): t[i-1, j-1], (i+1, j-1): t[i+1, j-1]}
    return {(i-1, j): t[i-1, j], (i+1, j): t[i+1, j], (i, j-1): t[i, j-1], (i, j+1): t[i, j+1], (i-1, j-1): t[i-1, j-1], (i-1, j+1): t[i-1, j+1], (i+1, j-1): t[i+1, j-1], (i+1, j+1): t[i+1, j+1]}

In [5]:
# Kiikszeli egy adott sor üres mezőit.
def row_9(t, i):
    t_copy = t.copy()
    t_copy[i][t_copy[i] == 0] = 9
    return t_copy

# Kiikszeli egy adott oszlop üres mezőit.
def col_9(t, j):
    t_copy = t.copy()
    t_copy[:,j][t_copy[:,j] == 0] = 9
    return t_copy

# Ha egy mező nem oldalszomszédos pókhálóval, akkor ide nem kerülhet pókháló,
# ezért kiikszeli azt.
def no_webs_near(t, i, j):
    t_copy = t.copy()
    if not 1 in get_4_neighbours(t_copy, i, j).values():
        t_copy[i, j] = 9
    return t_copy

In [6]:
# Ha egy kielégítetlen pókról egyértelmű, hogy csak egy pókhálóhoz tartozhat,
# akkor kielégítetté teszi őt és pókhálóját is. Ennek következtében előfordulhat,
# hogy elindul egy láncreakció, aminek során több pók-pókháló pár is kielégítetté válik;
# ezt a függvény rekurzív módon ismételt hívásával érjük el.

def check_satisfy_web(t, i, j):
    
    t_copy = t.copy()
    
    neighbours = get_4_neighbours(t_copy, i, j)
    
    if list(neighbours.values()).count(1) == 1:
        t_copy[i, j] = 4
        web_pos = list(neighbours.keys())[list(neighbours.values()).index(1)]   # a hozzá tartozó pókháló pozíciója
        t_copy[web_pos[0], web_pos[1]] = 2
        
        neighbours_of_web = get_4_neighbours(t_copy, web_pos[0], web_pos[1])
        
        if 3 in neighbours_of_web.values():    # a pókháló mellett van-e új kielégíthető pók?
            neighbour_pos = list(neighbours_of_web.keys())[list(neighbours_of_web.values()).index(3)]   # lekérjük annak a pozícióját
            t_copy = check_satisfy_web(t_copy, neighbour_pos[0], neighbour_pos[1])
            
    return t_copy

In [7]:
# Elhelyez egy kielégítetlen pókot a tábla (i,j)-edik mezőjére, kiikszeli ennek szomszédait,
# és update-eli, hogy a sorban és az oszlopban hány pók kell ezután (ha nulla, akkor
# kiikszeli az egész sort/oszlopot).

def put_spider(t, r, c, i, j):
    
    t_copy = t.copy()
    r_copy = r.copy()
    c_copy = c.copy()

    t_copy[i,j] = 3

    for key in get_8_neighbours(t_copy, i, j).keys():
        if t_copy[key[0], key[1]] == 0:
            t_copy[key[0], key[1]] = 9

    r_copy[i] -= 1
    c_copy[j] -= 1

    if r_copy[i] == 0:
        t_copy = row_9(t_copy, i)

    if c_copy[j] == 0:
        t_copy = col_9(t_copy, j)
    
    # Ha az újonnan elhelyezett pók kielégíthető, akkor a fentiek szerint jár el.
    t_copy = check_satisfy_web(t_copy, i, j)

    return t_copy, r_copy, c_copy

## A tábla definiálása

Az alábbi megoldható feladványokat a https://www.puzzle-tents.com/ weboldal segítségével generáltuk.

In [8]:
# 10x10-es példafeladványok

# easy
ex1_table = np.array([
    [0,1,0,0,0,0,0,0,0,1],
    [0,0,0,0,1,0,0,1,0,0],
    [0,0,1,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,0,0,1,0],
    [1,1,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,1,0,0,1,0],
    [1,0,0,0,1,0,0,0,0,0],
    [0,1,0,1,0,0,1,0,1,0],
    [1,0,0,0,0,0,0,0,0,0],
])
ex1_row = np.array([1,3,1,3,2,1,4,0,5,0])
ex1_col = np.array([4,0,4,0,3,1,2,2,1,3])


# easy
ex2_table = np.array([
    [0,1,0,0,0,0,0,0,1,0],
    [0,0,0,1,0,1,0,0,0,1],
    [0,1,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,1,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,1,0,1,1,0],
    [0,0,0,0,0,0,0,0,1,0],
    [1,0,0,0,0,0,0,1,0,1],
    [0,0,0,1,1,0,0,0,0,0],
])
ex2_row = np.array([3,1,3,0,3,1,2,3,0,4])
ex2_col = np.array([3,1,3,1,0,4,1,4,0,3])


# easy
ex3_table = np.array([
    [0,1,0,0,1,0,1,0,0,0],
    [1,0,0,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,1,0,0,0,0],
    [1,0,1,1,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0],
    [1,0,0,1,1,1,0,0,1,0],
    [0,0,0,0,0,0,0,0,0,0],
    [1,0,0,0,1,0,0,1,0,0],
    [0,0,0,0,0,0,0,0,1,0],
])
ex3_row = np.array([5,0,1,3,1,2,2,2,2,2])
ex3_col = np.array([3,1,3,0,3,2,2,2,2,2])


# hard
ex4_table = np.array([
    [0,1,0,0,0,0,0,1,1,0],
    [1,0,0,1,0,0,0,0,0,0],
    [0,0,0,1,1,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,0],
    [0,0,0,1,0,0,1,0,0,0],
    [0,1,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,1,0,1],
    [0,0,1,1,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,1],
    [0,0,0,1,0,0,0,1,0,0],
])
ex4_row = np.array([3,2,2,1,3,2,0,4,0,3])
ex4_col = np.array([2,3,1,2,3,1,1,2,2,3])


# hard
ex5_table = np.array([
    [0,1,0,0,0,0,1,0,0,0],
    [1,0,0,0,1,0,0,0,0,1],
    [0,1,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0,0,0],
    [1,0,1,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,0,0,1,0],
    [0,0,0,1,0,0,0,0,0,0],
    [1,0,1,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,1,0],
    [0,1,0,0,0,1,0,0,1,0],
])
ex5_row = np.array([4,1,2,2,0,4,0,3,1,3])
ex5_col = np.array([3,1,3,2,1,2,1,2,2,3])


# hard
ex6_table = np.array([
    [0,0,0,0,0,1,0,1,0,0],
    [0,0,1,0,0,0,0,0,0,0],
    [1,0,0,0,1,0,0,0,1,1],
    [0,0,0,0,1,0,0,1,0,0],
    [1,0,0,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,1,0,0,0],
    [0,0,1,1,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0,0],
    [0,0,0,1,0,1,0,0,0,0],
    [0,1,0,0,0,0,1,0,0,1],
])
ex6_row = np.array([3,2,1,2,2,3,1,1,1,4])
ex6_col = np.array([2,2,1,3,2,2,1,3,1,3])



In [9]:
# 6x6-os példafeladványok

# easy
ex7_table = np.array([
    [0,0,0,0,0,0],
    [0,1,0,1,0,0],
    [1,0,0,0,0,0],
    [0,0,0,0,0,1],
    [0,0,0,1,0,1],
    [0,0,1,0,0,0],
])
ex7_row = np.array([2,0,1,1,0,3])
ex7_col = np.array([0,3,0,2,1,1])


# easy
ex8_table = np.array([
    [1,0,0,0,0,0],
    [0,1,0,1,0,0],
    [1,0,0,0,0,0],
    [0,0,1,0,0,0],
    [0,1,0,0,0,0],
    [0,0,0,0,0,1],
])
ex8_row = np.array([0,3,0,2,0,2])
ex8_col = np.array([2,1,1,1,2,0])


# hard
ex9_table = np.array([
    [0,1,0,0,0,0],
    [0,0,0,1,0,0],
    [1,0,0,0,0,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,0],
    [0,1,0,1,0,1],
])
ex9_row = np.array([1,1,1,1,1,2])
ex9_col = np.array([1,1,2,1,1,1])

In [10]:
# egy 15x15-ös példafeladvány

ex10_table = np.array([
    [0,1,1,0,0,0,1,0,0,0,1,1,0,0,0],
    [0,1,0,0,0,0,0,1,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0,0,0,1,0],
    [0,0,0,0,1,1,0,0,0,0,1,1,0,0,1],
    [0,0,1,1,0,0,0,0,1,0,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
    [0,0,0,0,0,1,0,0,0,0,1,0,0,1,0],
    [0,0,1,0,0,0,0,0,1,0,0,0,0,0,0],
    [0,1,0,0,1,0,0,0,1,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0,1,0,1,0,0,0,0],
    [1,0,0,0,0,1,0,0,0,0,0,0,0,0,1],
    [0,1,0,0,0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
    [0,1,0,1,0,1,0,0,1,0,0,1,0,1,0]
])
ex10_row = [7,0,3,3,2,4,2,3,3,3,2,4,2,0,7]
ex10_col = [4,3,1,5,1,5,0,5,1,6,1,3,4,1,5]

In [11]:
# Random 10x10-es tábla generálása egy megoldhatatlan feladvány szemléltetéséhez

# ex11_table = np.array([[np.random.choice([0,1]) for i in range(10)] for j in range(10)])
# ex11_row = np.array([np.random.randint(0,5) for i in range(10)])
# ex11_col = np.array([np.random.randint(0,5) for i in range(10)])

# Mintázatok felismerése

A mintázatok elkódolásához reguláris kifejezéseket használunk.

In [12]:
import regex as re

In [17]:
# Visszaadja egy stringben az 1 hosszú nullás blokkok pozícióját.
def get_single_zeros(string):
    p = re.compile("^0[1-9]|[1-9]0[1-9]|[1-9]0$")
    match_indices = []
    for m in p.finditer(string, overlapped=True):
        if m.group()[0] == '0':
            match_indices.append(m.start())
        else:
            match_indices.append(m.start() + 1)
    return match_indices

# Visszaadja egy stringben a 3 hosszú nullás blokkok pozícióját.
def get_triple_zeros(string):
    p = re.compile("^000[1-9]|[1-9]000[1-9]|[1-9]000$")
    match_indices = []
    for m in p.finditer(string, overlapped=True):
        if m.group()[0] == '0':
            match_indices.append(m.start())
        else:
            match_indices.append(m.start() + 1)
    return match_indices

In [21]:
def search_for_patterns(t, r, c):
    t_copy = t.copy()
    r_copy = r.copy()
    c_copy = c.copy()
    
    made_changes = False
    
    # Ha egy sorban/oszlopban csak 1 és 2 hosszú blokkokban vannak az üres helyek,
    # és annyi blokk van, ahány pók még kell a sorba/oszlopba,
    # akkor az 1-es blokkokba beír pókokat.
    for i in range(N):
        row_string = "".join(t_copy[i].astype(str))

        single_zero_indices = get_single_zeros(row_string)

        p2 = re.compile("^00[1-9]|[1-9]00[1-9]|[1-9]00$")
        double_zero_matches = p2.findall(row_string, overlapped=True)

        if len(single_zero_indices) + 2*len(double_zero_matches) == row_string.count('0') and r_copy[i] == len(single_zero_indices) + len(double_zero_matches):
            for j in single_zero_indices:
                #print('yeah, spider!', i, j)
                t_copy, r_copy, c_copy = put_spider(t_copy, r_copy, c_copy, i, j)
                #print(t_copy, r_copy, c_copy)
                made_changes = True

    for j in range(N):
        col_string = "".join(t_copy[:,j].astype(str))

        single_zero_indices = get_single_zeros(col_string)

        p2 = re.compile("^00[1-9]|[1-9]00[1-9]|[1-9]00$")
        double_zero_matches = p2.findall(col_string, overlapped=True)

        if len(single_zero_indices) + 2*len(double_zero_matches) == col_string.count('0') and c_copy[j] == len(single_zero_indices) + len(double_zero_matches):
            for i in single_zero_indices:
                #print('yeah, spider!', i, j)
                t_copy, r_copy, c_copy = put_spider(t_copy, r_copy, c_copy, i, j)
                #print(t_copy, r_copy, c_copy)
                made_changes = True

    
    
    # Ha egy sorban/oszlopban csak 1 és 3 hosszú blokkokban vannak az üres helyek,
    # 1-es blokkok száma: x, 3-as blokkok száma: y,
    # a sorba/oszlopba beírandó pókok száma: z,
    # akkor ha x+2y=z,
    # akkor az 1-es blokkokba és a 3-as blokkokban a szélső helyekre beír pókokat.
    for i in range(N):
        row_string = "".join(t_copy[i].astype(str))

        single_zero_indices = get_single_zeros(row_string)
        triple_zero_indices = get_triple_zeros(row_string)

        if len(single_zero_indices) + 3*len(triple_zero_indices) == row_string.count('0') and r_copy[i] == len(single_zero_indices) + 2*len(triple_zero_indices):
            for j in single_zero_indices:
                #print('yeah, spider!')
                t_copy, r_copy, c_copy = put_spider(t_copy, r_copy, c_copy, i, j)
                #print(t_copy, r_copy, c_copy)
                made_changes = True
            for j in triple_zero_indices:
                #print('yeah, spider!')
                t_copy, r_copy, c_copy = put_spider(t_copy, r_copy, c_copy, i, j)
                t_copy, r_copy, c_copy = put_spider(t_copy, r_copy, c_copy, i, j+2)
                #print(t_copy, r_copy, c_copy)
                made_changes = True

    for j in range(N):
        col_string = "".join(t_copy[:,j].astype(str))

        single_zero_indices = get_single_zeros(col_string)
        triple_zero_indices = get_triple_zeros(col_string)

        if len(single_zero_indices) + 3*len(triple_zero_indices) == col_string.count('0') and c_copy[j] == len(single_zero_indices) + 2*len(triple_zero_indices):
            for i in single_zero_indices:
                #print('yeah, spider!')
                t_copy, r_copy, c_copy = put_spider(t_copy, r_copy, c_copy, i, j)
                #print(t_copy, r_copy, c_copy)
                made_changes = True
            for i in triple_zero_indices:
                #print('yeah, spider!')
                t_copy, r_copy, c_copy = put_spider(t_copy, r_copy, c_copy, i, j)
                t_copy, r_copy, c_copy = put_spider(t_copy, r_copy, c_copy, i+2, j)
                #print(t_copy, r_copy, c_copy)
                made_changes = True
    
    if made_changes:
        t_copy, r_copy, c_copy = search_for_patterns(t_copy, r_copy, c_copy)
    
    return t_copy, r_copy, c_copy
    

# A DFS implementálása

In [22]:
# Azért, hogy a backtracking hatékony legyen, mindig olyan helyre próbálunk meg egy pókot berakni,
# ahol a legkevesebb lehetőség közül kell választanunk. Az alábbi függvény megkeresi,
# hogy melyik sorban vagy oszlopban van a legkevesebb üres mező, vagy esetleg egy olyan pókháló körül,
# amely körül nincs kielégítetlen pók.
def min_options(t):
    min_type = '' # sor, oszlop vagy pókháló
    min_num = np.infty # mennyi a legkevesebb üres mező
    min_array = [] # ezen üres mezők pozíciója

    # sorok
    for i in range(N):
        options = np.sum(t[i,:]==0)
        if 0 < options and options < min_num:
            min_type = 'r'
            min_num = options
            min_array = []
            for j in range(N):
                if t[i,j] == 0:
                    min_array.append((i,j))
            #print(f"found a (so far) min row! the {i}th row has {min_num} options, namely at {min_array}")

    # oszlopok
    for j in range(N):
        options = np.sum(t[:,j]==0)
        if 0 < options and options < min_num:
            min_type = 'c'
            min_num = options
            min_array = []
            for i in range(N):
                if t[i,j] == 0:
                    min_array.append((i,j))
            #print(f"found a (so far) min column! the {j}th column has {min_num} options, namely at {min_array}")

    # pókhálók
    for i in range(N):
        for j in range(N):
            if t[i,j] == 1 and not 3 in get_4_neighbours(t, i, j).values():
                d = get_4_neighbours(t,i,j)
                options = len(d)
                for pos, val in d.items():
                    if val != 0:
                        options = options - 1
                if 0 < options and options < min_num:
                    min_type = 'w'
                    min_num = options
                    min_array = []
                    for pos, val in d.items():
                        if val == 0:
                            min_array.append(pos)
                    #print(f"found a (so far) min web! the web at ({i},{j}) has {min_num} options, namely at {min_array}")

    return min_type, min_num, min_array

In [23]:
N = 10 # itt kell változtatni N értékét

# ex1-6: N = 10
# ex7-9: N = 6
# ex10: N = 15

table = ex1_table
row = ex1_row
col = ex1_col

# A pókhálóval nem oldalszomszédos mezőket kiikszeljük.
for i in range(N):
    for j in range(N):
        if table[i, j] == 0:
            table = no_webs_near(table, i, j)

# A nulla pókot tartalmazó sorokat és oszlopokat kiikszeljük.            
for i in range(N):
    if row[i] == 0:
        table = row_9(table, i)
    if col[i] == 0:
        table = col_9(table, i)

# DFS        
solved = False 
sol = []
stack = []
visited = set()

s = table_to_string(table, row, col)
stack.append(s)
while stack != []:
    #print("current stack is:", stack)
    s = stack.pop()
    if s not in visited:
        visited.add(s)
        [table, row, col] = string_to_table(s)
        #print("current table:")
        #print(tabulate.tabulate(table, tablefmt='fancy_grid'), row, col)
        [table, row, col] = search_for_patterns(table, row, col) # mintázatok keresése
        [min_type, min_num, min_array] = min_options(table)
        #print("least options:", min_type, min_num, min_array)
        if len(min_array) == 0:
            if 1 in table: # ha már nincs üres mező a táblán, de van még kielégítetlen pókháló
                continue # visszalép
            else: # ha már nincs üres mező a táblán, és minden pókháló kielégített
                solved = True # jó megoldás
                sol = table
                stack = []
        
        neighbours = []

        for pos in min_array:
            [t,r,c] = put_spider(table, row, col, pos[0], pos[1])
            if not 1 in t:
                solved = True
                sol = t
                stack = []
            neighbours.append(table_to_string(t,r,c))

        for neighbour in neighbours:
            if neighbour not in visited:
                stack.append(neighbour)

if solved:
    pretty_sol = sol.astype(str)
    pretty_sol[sol == 9] = ""
    pretty_sol[sol == 2] = "#" # más opció: "🕸"
    pretty_sol[sol == 4] = '•' # más opció: '🕷'
    print("A megoldás:")
    print(tabulate.tabulate(pretty_sol, tablefmt='fancy_grid'))
    # Számokkal is kiírja a megoldást:
    # print(tabulate.tabulate(table, tablefmt='fancy_grid'), row, col)
else:
    print("Ez a feladvány nem megoldható.")

A megoldás:
╒═══╤═══╤═══╤═══╤═══╤═══╤═══╤═══╤═══╤═══╕
│ • │ # │   │   │   │   │   │   │   │ # │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │ • │   │ # │   │ • │ # │   │ • │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │ # │   │ • │   │   │   │   │ # │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ • │   │   │   │   │   │   │ • │ # │ • │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ # │ # │ • │   │ # │ • │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │   │   │ • │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ • │   │ • │ # │ • │ # │ • │   │ # │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ # │   │   │   │ # │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ • │ # │ • │ # │ • │   │ # │ • │ # │ • │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│ # │   │   │   │   │   │   │   │   │   │
╘═══╧═══╧═══╧═══╧═══╧═══╧═══╧═══╧═══╧═══╛
